In [1]:
!pip install tensorflow-macos tensorflow-metal opencv-python mediapipe-silicon scikit-learn matplotlib

You should consider upgrading via the '/Users/rakazein/.pyenv/versions/3.8.16/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from keras.models import load_model

objc[14594]: Class CaptureDelegate is implemented in both /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x16a7765a0) and /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x1372c8860). One of the two will be used. Which one is undefined.
objc[14594]: Class CVWindow is implemented in both /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x16a7765f0) and /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x130fd8a68). One of the two will be used. Which one is undefined.
objc[14594]: Class CVView is implemented in both /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x16a776618) and /Users/rakazein/.pyenv/versions/3.8.16/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x130fd8a90). One of the two will be used. Which one i

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [4]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_DATA_V2') 

# Actions that we try to detect
actions = np.array(['a', 'b', 'c', 
                    'd', 'e', 'f', 
                    'g', 'h', 'i', 
                    'j', 'k', 'l', 
                    'm', 'n', 'o', 
                    'p', 'q', 'r', 
                    's', 't', 'u',
                    'v', 'w', 'x',
                    'y', 'z'
                   ])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [5]:
label_map = {label:num for num, label in enumerate(actions)}

In [6]:
label_map

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25}

In [7]:
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        print(sequence)

20
18
27
9
0
11
7
29
16
6
28
17
1
10
19
26
8
21
24
23
4
15
3
12
2
13
5
14
22
25
20
18
27
9
0
11
7
29
16
6
28
17
1
10
19
26
8
21
24
23
4
15
3
12
2
13
5
14
22
25
20
18
27
9
0
11
7
29
16
6
28
17
1
10
19
26
8
21
24
23
4
15
3
12
2
13
5
14
22
25
20
18
27
9
0
11
7
29
16
6
28
17
1
10
19
26
8
21
24
23
4
15
3
12
2
13
5
14
22
25
20
18
27
9
0
11
7
29
16
6
28
17
1
10
19
26
8
21
24
23
4
15
3
12
2
13
5
14
22
25
20
18
27
9
0
11
7
29
16
6
28
17
1
10
19
26
8
21
24
23
4
15
3
12
2
13
5
14
22
25
20
18
27
9
0
11
7
29
16
6
28
17
1
10
19
26
8
21
24
23
4
15
3
12
2
13
5
14
22
25
20
18
27
9
0
11
7
29
16
6
28
17
1
10
19
26
8
21
24
23
4
15
3
12
2
13
5
14
22
25
20
18
27
9
0
11
7
29
16
6
28
17
1
10
19
26
8
21
24
23
4
15
3
12
2
13
5
14
22
25
20
18
27
9
0
11
7
29
16
6
28
17
1
10
19
26
8
21
24
23
4
15
3
12
2
13
5
14
22
25
20
18
27
9
0
11
7
29
16
6
28
17
1
10
19
26
8
21
24
23
4
15
3
12
2
13
5
14
22
25
20
18
27
9
0
11
7
29
16
6
28
17
1
10
19
26
8
21
24
23
4
15
3
12
2
13
5
14
22
25
20
18
27
9
0
11
7
29
16
6
28
17
1
10
19


In [8]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [9]:
np.array(sequences).shape

(780, 30, 126)

In [10]:
np.array(labels).shape

(780,)

In [11]:
X = np.array(sequences)

In [12]:
X.shape

(780, 30, 126)

In [13]:
y = to_categorical(labels).astype(int)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [15]:
X_train.shape

(741, 30, 126)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [17]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [18]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

Metal device set to: Apple M1 Pro


2023-08-19 02:39:27.177612: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-19 02:39:27.177933: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [19]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [20]:
model.fit(X_train, y_train, epochs=400, callbacks=[tb_callback])

Epoch 1/400


2023-08-19 02:39:31.576301: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-19 02:39:32.839416: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 54s 2s/step - loss: 3.2493 - categorical_accuracy: 0.0540
Epoch 2/400
24/24 [==============================] - 50s 2s/step - loss: 3.1257 - categorical_accuracy: 0.0972
Epoch 3/400
24/24 [==============================] - 51s 2s/step - loss: 2.9655 - categorical_accuracy: 0.0999
Epoch 4/400
24/24 [==============================] - 50s 2s/step - loss: 2.9867 - categorical_accuracy: 0.1093
Epoch 5/400
24/24 [==============================] - 51s 2s/step - loss: 3.0260 - categorical_accuracy: 0.1093
Epoch 6/400
24/24 [==============================] - 51s 2s/step - loss: 2.5331 - categorical_accuracy: 0.1714
Epoch 7/400
24/24 [==============================] - 51s 2s/step - loss: 2.7052 - categorical_accuracy: 0.2186
Epoch 8/400
24/24 [==============================] - 51s 2s/step - loss: 2.9060 - categorical_accuracy: 0.1552
Epoch 9/400
24/24 [==============================] - 51s 2s/step - loss: 2.4996 - categorical_accuracy: 0.1350
Epoch 10/400


24/24 [==============================] - 49s 2s/step - loss: 0.0967 - categorical_accuracy: 0.9622
Epoch 147/400
24/24 [==============================] - 49s 2s/step - loss: 0.2605 - categorical_accuracy: 0.9123
Epoch 148/400
24/24 [==============================] - 49s 2s/step - loss: 0.0975 - categorical_accuracy: 0.9730
Epoch 149/400
24/24 [==============================] - 49s 2s/step - loss: 0.0392 - categorical_accuracy: 0.9919
Epoch 150/400
24/24 [==============================] - 49s 2s/step - loss: 0.0252 - categorical_accuracy: 0.9933
Epoch 151/400
24/24 [==============================] - 49s 2s/step - loss: 0.0335 - categorical_accuracy: 0.9892
Epoch 152/400
24/24 [==============================] - 49s 2s/step - loss: 0.0271 - categorical_accuracy: 0.9933
Epoch 153/400
24/24 [==============================] - 49s 2s/step - loss: 0.7572 - categorical_accuracy: 0.7841
Epoch 154/400
24/24 [==============================] - 50s 2s/step - loss: 0.5998 - categorical_accuracy: 0.84

24/24 [==============================] - 53s 2s/step - loss: 0.0204 - categorical_accuracy: 0.9960
Epoch 291/400
24/24 [==============================] - 53s 2s/step - loss: 0.0655 - categorical_accuracy: 0.9892
Epoch 292/400
24/24 [==============================] - 53s 2s/step - loss: 0.0528 - categorical_accuracy: 0.9798
Epoch 293/400
24/24 [==============================] - 53s 2s/step - loss: 0.0654 - categorical_accuracy: 0.9798
Epoch 294/400
24/24 [==============================] - 52s 2s/step - loss: 0.0200 - categorical_accuracy: 0.9960
Epoch 295/400
24/24 [==============================] - 53s 2s/step - loss: 0.0277 - categorical_accuracy: 0.9906
Epoch 296/400
24/24 [==============================] - 53s 2s/step - loss: 0.0820 - categorical_accuracy: 0.9744
Epoch 297/400
24/24 [==============================] - 53s 2s/step - loss: 0.0186 - categorical_accuracy: 0.9973
Epoch 298/400
24/24 [==============================] - 53s 2s/step - loss: 0.0097 - categorical_accuracy: 0.99

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            48896     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 26)                858       
                                                                 
Total params: 204,218
Trainable params: 204,218
Non-trai

In [22]:
res = model.predict(X_test)

2023-08-19 10:37:20.102121: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 234ms/step


In [23]:
actions[np.argmax(res[4])]

'x'

In [24]:
actions[np.argmax(y_test[4])]

'x'

In [25]:
model.save('action_v1-20.h5')

In [26]:
del model

In [10]:
model = load_model('action_v1-20.h5')

OSError: No file or directory found at action_v1-20.h5

In [28]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [29]:
yhat = model.predict(X_test)

2023-08-19 02:33:17.098221: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 215ms/step


In [30]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [31]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[37,  0],
        [ 0,  2]],

       [[38,  0],
        [ 0,  1]],

       [[35,  0],
        [ 1,  3]],

       [[38,  0],
        [ 0,  1]],

       [[38,  0],
        [ 0,  1]],

       [[38,  0],
        [ 0,  1]],

       [[38,  0],
        [ 0,  1]],

       [[36,  0],
        [ 0,  3]],

       [[35,  0],
        [ 0,  4]],

       [[36,  0],
        [ 0,  3]],

       [[37,  0],
        [ 0,  2]],

       [[37,  0],
        [ 0,  2]],

       [[38,  0],
        [ 0,  1]],

       [[37,  0],
        [ 0,  2]],

       [[37,  1],
        [ 0,  1]],

       [[37,  0],
        [ 0,  2]],

       [[36,  0],
        [ 0,  3]],

       [[37,  0],
        [ 0,  2]],

       [[38,  0],
        [ 0,  1]],

       [[37,  0],
        [ 0,  2]]])

In [32]:
accuracy_score(ytrue, yhat)

0.9743589743589743

In [33]:
from scipy import stats

In [34]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [35]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

NameError: name 'image' is not defined

<Figure size 1800x1800 with 0 Axes>

In [36]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [37]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [38]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [39]:
def draw_styled_landmarks(image, results):
    #Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [40]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [43]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

IndexError: list index out of range